In [1]:
import sys
import os

# Chemin absolu du répertoire racine du projet
project_root = '/home/conite/Documents/WORKSPACE/PROJECTS/BI-PROJECT/BANK-REVIEWS-INTELLIGENCE/'

# Ajouter le répertoire src au chemin de recherche des modules
src = os.path.join(project_root, '.')
sys.path.append(src)



In [2]:
import pandas as pd
import numpy as np
from src.data_concatenation.concatenate import build_macro_table
from src.data_preprocessing.preprocessing import preprocess_dataframe
from src.data_preprocessing.topificator import TopicExtractor
from datetime import datetime
from datetime import datetime, timedelta
from tqdm import tqdm

CURRENT_DATE = datetime.now().strftime("%Y-%m-%d")
DATA_PATH = os.path.join(project_root, 'data')
CITIES_PATH = os.path.join(project_root, 'data', 'utils', 'countries_cities-full.json')
RAW_SAVE_PATH = os.path.join(project_root, 'data', 'temp')
PARCKET_PATH = os.path.join(project_root, 'data', 'parquet', CURRENT_DATE)


concatenate_parquets = build_macro_table(DATA_PATH)

# --------------------------------------------------------
# 0.3 Preprocessing of concatenate parquets
# --------------------------------------------------------
# df = preprocess_dataframe(concatenate_parquets)



/home/conite/Documents/WORKSPACE/PROJECTS/BI-PROJECT/BANK-REVIEWS-INTELLIGENCE/data/parquet/2024-07-30/Morocco/Nador.parquet: 90 rows
Macro table saved at: /home/conite/Documents/WORKSPACE/PROJECTS/BI-PROJECT/BANK-REVIEWS-INTELLIGENCE/src/macro_table/macro_table.parquet
Macro table saved as CSV at : /home/conite/Documents/WORKSPACE/PROJECTS/BI-PROJECT/BANK-REVIEWS-INTELLIGENCE/src/macro_table/macro_table.csv


In [3]:
len(concatenate_parquets.columns)

17

In [5]:

# --------------------------------------------------------
# 0.3 Preprocessing of concatenate parquets
# --------------------------------------------------------
df = preprocess_dataframe(concatenate_parquets)

/home/conite/Documents/WORKSPACE/PROJECTS/BI-PROJECT/BANK-REVIEWS-INTELLIGENCE/./src/data_preprocessing/preprocessing.py:64: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna({


Topic : {'topics': [('Application mobile', 'Negative', ['Bug', 'Connexion difficile'])]}


  1%|▏         | 1/73 [01:18<1:33:56, 78.29s/it]

Topic 0 : Pays Ahsan ::: {'topics': [('Service client', 'Positive', ['réactif', 'rapidité']), ('Frais bancaires', 'Negative', ['élevés'])]}


  3%|▎         | 2/73 [02:15<1:18:12, 66.09s/it]

...

Topic 87 : Prestation : 0
Solutions : 0
Attention : 0 ::: {'topics': [('Sécurité', 'Negative', ['Alerte de sécurité non fonctionnelle'])]}


 99%|█████████▊| 72/73 [1:03:39<00:51, 51.81s/it]

Topic 88 : Très bien situé, personnel excellent. Il y a parfois des problèmes sur Internet ::: {'topics': [('Localisation et accessibilité', 'Positive', ['Bonne emplacement', 'Personnel excellent']), ('Sécurité', 'Negative', ['Problèmes sur Internet'])]}


100%|██████████| 73/73 [1:04:22<00:00, 52.91s/it]

Topic 89 : service 0 ::: {'topics': [('Service client', 'Negative', ['pas réactif', 'trop long'])]}


In [5]:
len(df.columns)

14

In [6]:
# all df
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df[['Reviewer_Text', 'Reviewer_Star']]


,Reviewer_Text,Reviewer_Star
0,Pays Ahsan,5
1,Berger,4
2,Expérience neutre,3
3,Très mauvaise expérience,1
4,Expérience neutre,3
5,Très bonne expérience,5
6,Expérience neutre,3
7,J'ai eu l'honneur d'être accueilli chaleureuse...,5
8,Siège de la banque populaire Nador Al hoceima.,5
9,Bon accueil et professionnel,5


In [8]:
df.isna().sum()

Country                      0
Town                         0
Bank_Name                    0
Bank_Phone_number            0
Bank_Address                 0
Bank_Website                 0
Reviewer_Name                0
Reviewer_Star                0
Reviewer_Text                0
Reviewer_Publish_Date        0
Reviewer_Like_Reaction       0
Reviewer_Profile_Link        0
Reviewer_Owner_Reply         1
Reviewer_Owner_Reply_Date    0
dtype: int64

In [9]:
df.head()

,Country,Town,Bank_Name,Bank_Phone_number,Bank_Address,Bank_Website,Reviewer_Name,Reviewer_Star,Reviewer_Text,Reviewer_Publish_Date,Reviewer_Like_Reaction,Reviewer_Profile_Link,Reviewer_Owner_Reply,Reviewer_Owner_Reply_Date
0,Morocco,Nador,Banque Populaire,Not available,"53Hg+Fvc, Bd Mohamed Ameziane Errifi, Nador",not available,Habib Chouaa,5,Pays Ahsan,2016-08-08 09:34:58.486197,0,https://www.google.com/maps/contrib/1006948281...,Nothing,1970-01-01
1,Morocco,Nador,Banque Populaire,Not available,"53Hg+Fvc, Bd Mohamed Ameziane Errifi, Nador",not available,Mohamed Nador,4,Berger,2016-08-08 09:34:58.486214,0,https://www.google.com/maps/contrib/1165746157...,Nothing,1970-01-01
2,Morocco,Nador,Banque Populaire,Not available,"53Hg+Fvc, Bd Mohamed Ameziane Errifi, Nador",not available,Hafaimya,3,Expérience neutre,2021-08-07 09:34:58.486222,0,https://www.google.com/maps/contrib/1027703754...,Nothing,1970-01-01
3,Morocco,Nador,Banque Populaire,Not available,"53Hg+Fvc, Bd Mohamed Ameziane Errifi, Nador",not available,Imad Charkani El-Hassani,1,Très mauvaise expérience,2018-08-08 09:34:58.486229,0,https://www.google.com/maps/contrib/1029760643...,Nothing,1970-01-01
4,Morocco,Nador,Banque Populaire,Not available,"53Hg+Fvc, Bd Mohamed Ameziane Errifi, Nador",not available,Abdelkarim Ajarray,3,Expérience neutre,2017-08-08 09:34:58.486236,0,https://www.google.com/maps/contrib/1158693667...,Nothing,1970-01-01


In [ ]:


for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    review = row['Reviewer_Text']
    
    try:
        topics = extractor.extract(review, type='SINGLE_SOURCE')
        topics_array = topics['topics']
    
        if len(topics_array) == 0:
            static_topic, static_sentiment, static_sub_topic = generate_static_topics_and_sentiments(row['Reviewer_Sart'])
            new_row = row.copy()
            new_row['Topic'] = static_topic
            new_row['Sentiment'] = static_sentiment
            new_row['Sub_Topic'] = static_sub_topic
            df_macro = pd.concat([df_macro, pd.DataFrame([new_row])], ignore_index=True)
            row_accumulated += 1
        else:
            for tuple_ in topics_array:
                new_row = row.copy()
                new_row['Topic'] = tuple_[0]
                new_row['Sentiment'] = tuple_[1]
                new_row['Sub_Topic'] = tuple_[2]
                df_macro = pd.concat([df_macro, pd.DataFrame([new_row])], ignore_index=True)
                row_accumulated += 1
            
        # Sauvegarder à chaque intervalle défini
        if row_accumulated >= save_interval:
            df_macro.to_csv(output_file, index=False)
            row_accumulated = 0
    except Exception as e:
        print(f"Erreur rencontrée à l'index {index}: {e}")
        pass